# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import matplotlib.pyplot as plt
import os
import json

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,waitangi,-43.9535,-176.5597,16.01,94,96,4.02,NZ,1712779994
1,1,iskateley,67.6803,53.1512,-13.88,96,100,2.68,RU,1712779994
2,2,wuxi,31.5689,120.2886,13.44,83,100,3.32,CN,1712779994
3,3,sinabang,2.4803,96.3801,26.29,87,96,2.18,ID,1712779995
4,4,port-aux-francais,-49.3500,70.2167,7.62,99,100,4.52,TF,1712779995


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display
# Configure the map plot
coordinates = {
    "Lat": (city_data_df["Lat"]),
    "Lng": (city_data_df["Lng"]),
    "City": (city_data_df["City"]),
    "Humidity": (city_data_df["Humidity"])
}

coordinates_df = pd.DataFrame(coordinates)

humidity_map = coordinates_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color = "City"
)
# Display the map
humidity_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
# Drop any rows with null values
ideal_city = city_data_df.loc[(city_data_df["Max Temp"] >= 25) & 
                            (city_data_df["Max Temp"] < 30).dropna()]

# Display sample data
display(ideal_city)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,sinabang,2.4803,96.3801,26.29,87,96,2.18,ID,1712779995
11,11,le port,-20.9373,55.2919,25.00,77,36,3.24,RE,1712779996
13,13,al ghayzah,16.2079,52.1760,26.77,73,2,3.82,YE,1712779699
14,14,laguna,38.4210,-121.4238,25.72,44,0,3.60,US,1712779996
20,20,ambon,-3.6954,128.1814,25.08,95,40,1.03,ID,1712779997
...,...,...,...,...,...,...,...,...,...,...
539,539,ulu tiram,1.6000,103.8167,26.92,94,32,1.57,MY,1712780070
541,541,arica,-18.4750,-70.3042,26.75,73,59,2.77,CL,1712779922
547,547,belo sur tsiribihina,-19.7000,44.5500,25.43,84,42,0.98,MG,1712780071
550,550,siddhapur,23.9167,72.3833,28.86,26,100,2.54,IN,1712780071


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
3,sinabang,ID,2.4803,96.3801,87,
11,le port,RE,-20.9373,55.2919,77,
13,al ghayzah,YE,16.2079,52.1760,73,
14,laguna,US,38.4210,-121.4238,44,
20,ambon,ID,-3.6954,128.1814,95,
...,...,...,...,...,...,...
539,ulu tiram,MY,1.6000,103.8167,94,
541,arica,CL,-18.4750,-70.3042,73,
547,belo sur tsiribihina,MG,-19.7000,44.5500,84,
550,siddhapur,IN,23.9167,72.3833,26,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories":"accommodation.hotel",
            "limit": 20,
            "apiKey":geoapify_key    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
sinabang - nearest hotel: No hotel found
le port - nearest hotel: No hotel found
al ghayzah - nearest hotel: فندق تاج العرب
laguna - nearest hotel: Holiday Inn Express & Suites
ambon - nearest hotel: Hotel Hero
west island - nearest hotel: Cocos Village Bungalows
karratha - nearest hotel: Karratha International Hotel
todos santos - nearest hotel: Hotel Casa Tota
flying fish cove - nearest hotel: Christmas Island Lodge
bonthe - nearest hotel: No hotel found
lihue - nearest hotel: Kauai Palms
fort bragg - nearest hotel: Airborne Inn Lodging
hithadhoo - nearest hotel: Pebbles Inn
kailua-kona - nearest hotel: Kona Seaside Hotel
karanja - nearest hotel: No hotel found
uturoa - nearest hotel: Hawaiki Nui hotel
georgetown - nearest hotel: Page 63 hostel
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
afaahiti - nearest hotel: Omati Lodge
visakhapatnam - nearest hotel: Fairfield by Marriot
barreiro do jaiba - nearest hotel: No hotel found
mahibadhoo 

,City,Country,Lat,Lng,Humidity,Hotel Name
3,sinabang,ID,2.4803,96.3801,87,No hotel found
11,le port,RE,-20.9373,55.2919,77,No hotel found
13,al ghayzah,YE,16.2079,52.1760,73,فندق تاج العرب
14,laguna,US,38.4210,-121.4238,44,Holiday Inn Express & Suites
20,ambon,ID,-3.6954,128.1814,95,Hotel Hero
...,...,...,...,...,...,...
539,ulu tiram,MY,1.6000,103.8167,94,Lading 31
541,arica,CL,-18.4750,-70.3042,73,Hotel Andalucía
547,belo sur tsiribihina,MG,-19.7000,44.5500,84,Karibo
550,siddhapur,IN,23.9167,72.3833,26,Hotel amar deep


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    scale = 1,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)